### Reading MNIST

In [1]:
import pandas as pd
import numpy as np


train = pd.read_csv("./data/mnist_train.csv")
test = pd.read_csv("./data/mnist_test.csv")

# Reestructuring
train_label, train_image = train["label"].to_numpy(), train[[x for x in train.columns if x != "label"]].to_numpy().reshape((-1, 1, 28, 28))/255
test_label, test_image = train["label"].to_numpy(), train[[x for x in train.columns if x != "label"]].to_numpy().reshape((-1, 1, 28, 28))/255

# One-hotting
one_hot = np.zeros((len(train_label), 10))
for label in range(train_label.shape[0]):
    one_hot[label, train_label[label]] = 1
train_label = one_hot

one_hot = np.zeros((len(test_label), 10))
for label in range(test_label.shape[0]):
    one_hot[label, test_label[label]] = 1
test_label = one_hot

FileNotFoundError: [Errno 2] No such file or directory: './data/mnist_train.csv'

### Creating The Model

In [1]:
import framework as fm

class Model(fm.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = fm.Conv2d(1, 6, 5, 1, bias = False)
        self.relu1 = fm.ReLu()
        self.maxpool1 = fm.MaxPool2d(2, 6, 2)

        self.conv2 = fm.Conv2d(6, 16, 5, 1, bias = False)
        self.maxpool2 = fm.MaxPool2d(2, 16, 2)

        self.lin1 = fm.Linear(256, 84, bias=False)
        self.dropout1 = fm.Dropout(0.5)
        self.lin2 = fm.Linear(84, 10, bias=False)


    def forward(self, x):
        x = self.conv1.forward(x) 
        x = self.relu1.forward(x) 
        x = self.maxpool1.forward(x)
        x = self.conv2.forward(x)
        x = self.maxpool2.forward(x)
        x = x.reshape((-1, 256))
        x = self.lin1.forward(x) 
        x = self.dropout1.forward(x)
        x = self.lin2.forward(x)
        return x

model = Model()
model.train()
loss_fn = fm.CrossEntropy()
optim = fm.Adadelta(model.get_parameters())


#### Training The Model

In [6]:
batch_size = 128
num_batches = int(len(train_label)/batch_size)
epochs = 10


for epoch in range(epochs):

    print(f"Epoch {epoch+1}\n-------------------------------")

    total_loss = 0
    correct = 0

    for batch in range(num_batches):
        batch_start, batch_end = (batch * batch_size, (batch + 1) * batch_size)

        inpt = fm.Tensor(train_image[batch_start:batch_end])
        label = fm.Tensor(train_label[batch_start:batch_end])
        
        pred = model.forward(inpt)

        pred_nums = np.argmax(pred.data, axis=1)
        label_nums = np.argmax(label.data, axis=1)

        correct += (pred_nums == label_nums).sum()
        
        loss = loss_fn.forward(pred, label)

        total_loss += loss 

        if batch % int(num_batches/10) == 0:
            print(f"loss: {loss} [{(batch+1)*batch_size}/{batch_size*num_batches}]")   
            correct = 0
      
        loss_fn.backward()

        optim.step()

    print("Avg error: ", total_loss / num_batches, "\n")  

Epoch 1
-------------------------------
loss: 7.092289924621582 [128/59904]
loss: 1.0646944046020508 [6016/59904]
loss: 0.9254031181335449 [11904/59904]
loss: 0.7850354313850403 [17792/59904]
loss: 0.40042510628700256 [23680/59904]
loss: 0.4397575855255127 [29568/59904]
loss: 0.34302476048469543 [35456/59904]
loss: 0.6271507740020752 [41344/59904]
loss: 0.29360413551330566 [47232/59904]
loss: 0.20686164498329163 [53120/59904]
loss: 0.037199005484580994 [59008/59904]
Avg error:  0.64735186 

Epoch 2
-------------------------------
loss: 0.32301923632621765 [128/59904]
loss: 0.2716735899448395 [6016/59904]
loss: 0.2753898799419403 [11904/59904]
loss: 0.42396280169487 [17792/59904]
loss: 0.22107890248298645 [23680/59904]
loss: 0.2898280620574951 [29568/59904]
loss: 0.21280376613140106 [35456/59904]
loss: 0.39603757858276367 [41344/59904]
loss: 0.1817241609096527 [47232/59904]
loss: 0.14322152733802795 [53120/59904]
loss: 0.01380546297878027 [59008/59904]
Avg error:  0.23369384 

Epoch 3
-

/home/eric/Documents/AI_FRAMEWORK/NN/framework.py:363: RuntimeWarning: divide by zero encountered in log
  loss = -(np.log(softmax) * y.data).sum(1).mean()
/home/eric/Documents/AI_FRAMEWORK/NN/framework.py:363: RuntimeWarning: invalid value encountered in multiply
  loss = -(np.log(softmax) * y.data).sum(1).mean()


loss: 0.01713491976261139 [35456/59904]
loss: 0.1531791090965271 [41344/59904]
loss: 0.048473745584487915 [47232/59904]
loss: 0.0558614507317543 [53120/59904]
loss: 0.01211558561772108 [59008/59904]
Avg error:  nan 



### Saving model

In [16]:
model.save("97_71precision")

Object saved successfully.


### Load Model

In [ ]:
import pickle
import framework as fm 
import numpy as np 
import pandas as np

with open('97precision.pkl', 'rb') as file:
    model = pickle.load(file)

print("Object loaded successfully.")

Object loaded successfully.


#### Test Accuracy

In [7]:
# Testing
model.test()
correct = 0
batch_size = 128
num_batches = int(len(train_label)/batch_size)
for batch in range(num_batches):
        batch_start, batch_end = (batch * batch_size, (batch + 1) * batch_size)

        inpt = fm.Tensor(test_image[batch_start:batch_end])
        label = fm.Tensor(test_label[batch_start:batch_end])
        
        pred = model.forward(inpt)

        pred_nums = np.argmax(pred.data, axis=1)
        label_nums = np.argmax(label.data, axis=1)

        correct += (pred_nums == label_nums).sum()

print("Accuracy:", correct / (batch_size * num_batches) * 100, "%")

Accuracy: 96.79153311965813 %


#### Creation of API

In [ ]:
import uvicorn
from fastapi import Request, FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/predict")
async def predict(request: Request):
    res = await request.json()
    picture = fm.Tensor(np.array(res).reshape((1, 1, 28, 28)))
    pred = model.forward(picture)
    num = int(np.argmax(pred.data, axis=1))

    return num

if __name__ == "__main__":
    
    nest_asyncio.apply()  # Allow asyncio.run to work in a running loop
    uvicorn.run(app, host="127.0.0.1", port=3838)

INFO:     Started server process [32010]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3838 (Press CTRL+C to quit)


INFO:     127.0.0.1:34164 - "OPTIONS /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:34164 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:40308 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:40308 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:40308 - "POST /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:40308 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [32010]


In [2]:
import numpy as np
import framework as fm

np.random.seed(0)
data = fm.Tensor(np.random.random((1, 4, 3)))
drop = fm.BatchNorm(axis=2)
drop.training = True

res = drop.forward(data)

loss = fm.Tensor(np.ones((data.data.shape)))

print("Data: ", data)

print("Res: ", res)
print("Max:", data.data.max(), "Min:", data.data.min(), "Var:", data.data.var(axis=2), "Mean:", data.data.mean(axis=2))

res.backward(loss)

print("Data Loss: ", data.gradient)

Data:  [[[0.5488135  0.71518934 0.60276335]
  [0.5448832  0.4236548  0.6458941 ]
  [0.4375872  0.891773   0.96366274]
  [0.3834415  0.79172504 0.5288949 ]]]
Res:  [[[-1.0585594   1.33951    -0.2809497 ]
  [ 0.0741307  -1.2593846   1.1852533 ]
  [-1.4028116   0.54708767  0.855723  ]
  [-1.0922413   1.3237654  -0.2315248 ]]]
Max: 0.96366274 Min: 0.3834415 Var: [[0.00480346 0.00825443 0.05424537 0.02854798]] Mean: [[0.6222554  0.53814405 0.76434106 0.5680205 ]]
Data Loss:  [[[14.413569  14.413569  14.413569 ]
  [11.000024  11.000024  11.000024 ]
  [ 4.2931757  4.2931757  4.2931757]
  [ 5.917473   5.917473   5.917473 ]]]
